# RAG-Based Document Extraction Pipeline

This notebook implements:
1. Markdown-based chunking
2. LLM tagging and metadata extraction
3. Vector storage with ChromaDB
4. Hybrid retrieval (Semantic + BM25)
5. Final extraction using RAG

In [ ]:
# Prerequisites: pip install chromadb rank-bm25 groq docling
import os
os.environ["GROQ_KEY"] = "your_groq_api_key_here"

In [ ]:
# Convert PDF to markdown using Docling
from docling.document_converter import DocumentConverter

source = r"C:\Users\madha\OneDrive\Documents\PDF Data Extraction Pipeline\Data\s12887-020-02231-5.pdf"
converter = DocumentConverter()
doc = converter.convert(source).document

print("Document converted successfully!")

In [ ]:
# Step 1: Advanced chunking strategy
import re
from typing import List, Dict

def extract_tables(markdown_text: str) -> List[Dict[str, any]]:
    """Extract tables from markdown and return them with their positions."""
    tables = []
    # Pattern to match markdown tables
    table_pattern = r'(\|[^\n]+\|[\n\r]+\|[-:\s|]+\|[\n\r]+(?:\|[^\n]+\|[\n\r]+)*)'
    
    for match in re.finditer(table_pattern, markdown_text):
        tables.append({
            'content': match.group(0),
            'start': match.start(),
            'end': match.end()
        })
    
    return tables

def recursive_chunk_with_overlap(text: str, max_size: int = 8000, overlap: int = 200) -> List[str]:
    """
    Recursively split text into chunks with overlap.
    Tries to split at paragraph, then sentence, then word boundaries.
    """
    if len(text) <= max_size:
        return [text]
    
    chunks = []
    
    # Try splitting by paragraphs first
    paragraphs = text.split('\n\n')
    if len(paragraphs) > 1:
        current_chunk = ""
        
        for para in paragraphs:
            if len(current_chunk) + len(para) + 2 <= max_size:
                current_chunk += ("\n\n" if current_chunk else "") + para
            else:
                if current_chunk:
                    chunks.append(current_chunk)
                    # Add overlap from end of previous chunk
                    overlap_text = current_chunk[-overlap:] if len(current_chunk) > overlap else current_chunk
                    current_chunk = overlap_text + "\n\n" + para
                else:
                    # Single paragraph is too large, need to split further
                    sub_chunks = recursive_chunk_with_overlap(para, max_size, overlap)
                    chunks.extend(sub_chunks[:-1])
                    current_chunk = sub_chunks[-1] if sub_chunks else ""
        
        if current_chunk:
            chunks.append(current_chunk)
        
        return chunks
    
    # Try splitting by sentences
    sentences = re.split(r'(?<=[.!?])\s+', text)
    if len(sentences) > 1:
        current_chunk = ""
        
        for sent in sentences:
            if len(current_chunk) + len(sent) + 1 <= max_size:
                current_chunk += (" " if current_chunk else "") + sent
            else:
                if current_chunk:
                    chunks.append(current_chunk)
                    overlap_text = current_chunk[-overlap:] if len(current_chunk) > overlap else current_chunk
                    current_chunk = overlap_text + " " + sent
                else:
                    # Single sentence too large, split by words
                    sub_chunks = recursive_chunk_with_overlap(sent, max_size, overlap)
                    chunks.extend(sub_chunks[:-1])
                    current_chunk = sub_chunks[-1] if sub_chunks else ""
        
        if current_chunk:
            chunks.append(current_chunk)
        
        return chunks
    
    # Last resort: split by words
    words = text.split()
    current_chunk = ""
    
    for word in words:
        if len(current_chunk) + len(word) + 1 <= max_size:
            current_chunk += (" " if current_chunk else "") + word
        else:
            if current_chunk:
                chunks.append(current_chunk)
                overlap_text = current_chunk[-overlap:] if len(current_chunk) > overlap else current_chunk
                current_chunk = overlap_text + " " + word
            else:
                # Single word exceeds max_size (rare edge case)
                chunks.append(word[:max_size])
                current_chunk = word[max_size:]
    
    if current_chunk:
        chunks.append(current_chunk)
    
    return chunks

def split_large_chunk(chunk_content: str, header: str, max_size: int, overlap: int = 200) -> List[Dict[str, str]]:
    """Split a large chunk at paragraph boundaries with overlap."""
    split_texts = recursive_chunk_with_overlap(chunk_content, max_size, overlap)
    
    split_chunks = []
    for i, text in enumerate(split_texts, 1):
        header_text = f"{header} (Part {i})" if len(split_texts) > 1 else header
        split_chunks.append({
            "header": header_text,
            "content": text.strip(),
            "char_count": len(text.strip()),
            "contains_table": False
        })
    
    return split_chunks

def chunk_by_markdown_headers(markdown_text: str, 
                               min_chunk_size: int = 1000, 
                               max_chunk_size: int = 8000,
                               overlap: int = 200) -> List[Dict[str, str]]:
    """
    Advanced chunking strategy:
    1. Extract tables as separate chunks
    2. Split by headers (## and ###) if present
    3. If no headers, use recursive chunking with overlap
    4. Combine small chunks (< min_chunk_size)
    5. Split large chunks (> max_chunk_size) at paragraph boundaries with overlap
    """
    
    # Extract tables first
    tables = extract_tables(markdown_text)
    table_ranges = [(t['start'], t['end']) for t in tables]
    
    # Split by headers
    pattern = r'(^#{2,3}\s+.+?)(?=\n#{2,3}\s+|\Z)'
    matches = list(re.finditer(pattern, markdown_text, re.MULTILINE | re.DOTALL))
    
    # If no markdown headers found, use recursive chunking
    if not matches:
        print("No markdown headers found. Using recursive chunking with overlap...")
        chunk_texts = recursive_chunk_with_overlap(markdown_text, max_chunk_size, overlap)
        return [{
            "header": f"Chunk {i+1}",
            "content": text,
            "char_count": len(text),
            "contains_table": False
        } for i, text in enumerate(chunk_texts)]
    
    raw_chunks = []
    for match in matches:
        chunk_text = match.group(1).strip()
        if chunk_text and len(chunk_text) > 50:
            header_match = re.match(r'^(#{2,3}\s+)(.+)', chunk_text)
            header = header_match.group(2).strip() if header_match else "Unknown"
            
            # Check if this chunk contains a table
            chunk_start = match.start()
            chunk_end = match.end()
            contains_table = any(
                (chunk_start <= t_start < chunk_end) or (chunk_start < t_end <= chunk_end)
                for t_start, t_end in table_ranges
            )
            
            raw_chunks.append({
                "header": header,
                "content": chunk_text,
                "char_count": len(chunk_text),
                "contains_table": contains_table,
                "position": len(raw_chunks)
            })
    
    # Process chunks: combine small, split large
    processed_chunks = []
    i = 0
    
    while i < len(raw_chunks):
        current_chunk = raw_chunks[i]
        
        # If chunk contains a table, keep it as-is (don't combine or split)
        if current_chunk['contains_table']:
            processed_chunks.append(current_chunk)
            i += 1
            continue
        
        # If chunk is too small, combine with next chunks
        if current_chunk['char_count'] < min_chunk_size:
            combined_content = current_chunk['content']
            combined_header = current_chunk['header']
            j = i + 1
            
            # Keep combining until we reach min_chunk_size or hit a table
            while j < len(raw_chunks) and len(combined_content) < min_chunk_size:
                if raw_chunks[j]['contains_table']:
                    break
                combined_content += "\n\n" + raw_chunks[j]['content']
                combined_header += " + " + raw_chunks[j]['header']
                j += 1
            
            # After combining, check if it's now too large and needs splitting
            if len(combined_content) > max_chunk_size:
                split_results = split_large_chunk(combined_content, combined_header, max_chunk_size, overlap)
                processed_chunks.extend(split_results)
            else:
                processed_chunks.append({
                    "header": combined_header,
                    "content": combined_content,
                    "char_count": len(combined_content),
                    "contains_table": False
                })
            i = j
            continue
        
        # If chunk is too large, split at paragraph boundaries
        if current_chunk['char_count'] > max_chunk_size:
            split_results = split_large_chunk(
                current_chunk['content'], 
                current_chunk['header'], 
                max_chunk_size,
                overlap
            )
            processed_chunks.extend(split_results)
            i += 1
            continue
        
        # Chunk is just right
        processed_chunks.append(current_chunk)
        i += 1
    
    return processed_chunks

markdown_content = doc.export_to_markdown()
chunks = chunk_by_markdown_headers(markdown_content, min_chunk_size=1000, max_chunk_size=8000, overlap=200)

print(f"Created {len(chunks)} chunks")
print(f"\nChunk size distribution:")
small = sum(1 for c in chunks if c['char_count'] < 1000)
medium = sum(1 for c in chunks if 1000 <= c['char_count'] <= 8000)
large = sum(1 for c in chunks if c['char_count'] > 8000)
tables = sum(1 for c in chunks if c.get('contains_table', False))

print(f"  Small (<1000): {small}")
print(f"  Medium (1000-8000): {medium}")
print(f"  Large (>8000): {large}")
print(f"  Contains tables: {tables}")

print(f"\nFirst 5 chunks:")
for i, chunk in enumerate(chunks[:5]):
    table_marker = " [TABLE]" if chunk.get('contains_table', False) else ""
    print(f"  {i+1}. {chunk['header'][:60]}... ({chunk['char_count']} chars){table_marker}")

if large > 0:
    print(f"\n⚠️ WARNING: {large} chunks still exceed max size (8000 chars)")
    print("Large chunks (showing headers):")
    for i, chunk in enumerate(chunks):
        if chunk['char_count'] > 8000:
            print(f"  - {chunk['header'][:80]}... ({chunk['char_count']} chars)")

In [ ]:
# Step 2: Tag chunks with LLM and extract metadata
from groq import Groq

groq_client = Groq(api_key=os.environ["GROQ_KEY"])

tagging_prompt = """Analyze the following text chunk from a research paper and:

1. Assign relevant tags (comma-separated):
   - <summary>: abstract, introduction, conclusion sections
   - <research_methods>: methodology, study design, data collection
   - <findings_conclusion>: results, findings, conclusions
   - <metadata>: author names, publication date, affiliations

2. Extract metadata if present:
   - Authors: (list if found, else "None")
   - Date: (date if found, else "None")

Format:
TAGS: <tag1>, <tag2>
AUTHORS: names or None
DATE: date or None

Text:
{chunk_text}"""

document_metadata = {"authors": None, "date": None}
tagged_chunks = []

print("Tagging chunks with LLM...")
for i, chunk in enumerate(chunks):
    print(f"  Processing {i+1}/{len(chunks)}...", end='\r')
    
    prompt = tagging_prompt.format(chunk_text=chunk['content'][:2000])
    
    try:
        response = groq_client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.1,
            max_completion_tokens=200,
        )
        
        result = response.choices[0].message.content
        tags = []
        
        for line in result.split('\n'):
            if line.startswith('TAGS:'):
                tags = [t.strip() for t in line.replace('TAGS:', '').strip().split(',')]
            elif line.startswith('AUTHORS:'):
                authors = line.replace('AUTHORS:', '').strip()
                if authors.lower() != 'none' and not document_metadata["authors"]:
                    document_metadata["authors"] = authors
            elif line.startswith('DATE:'):
                date = line.replace('DATE:', '').strip()
                if date.lower() != 'none' and not document_metadata["date"]:
                    document_metadata["date"] = date
        
        tagged_chunks.append({**chunk, "tags": tags, "chunk_id": i})
    except Exception as e:
        print(f"\n  Error on chunk {i}: {e}")
        tagged_chunks.append({**chunk, "tags": [], "chunk_id": i})

print(f"\n\nMetadata extracted:")
print(f"  Authors: {document_metadata['authors']}")
print(f"  Date: {document_metadata['date']}")

In [ ]:
# Step 3: Store in ChromaDB with embeddings
import chromadb
from chromadb.utils import embedding_functions

chroma_client = chromadb.Client()
collection = chroma_client.get_or_create_collection(
    name="research_papers",
    embedding_function=embedding_functions.DefaultEmbeddingFunction()
)

documents = [chunk['content'] for chunk in tagged_chunks]
metadatas = [{
    "header": chunk['header'],
    "tags": ','.join(chunk['tags']),
    "char_count": chunk['char_count'],
    "chunk_id": chunk['chunk_id']
} for chunk in tagged_chunks]
ids = [f"chunk_{chunk['chunk_id']}" for chunk in tagged_chunks]

collection.add(documents=documents, metadatas=metadatas, ids=ids)

print(f"Stored {len(documents)} chunks in ChromaDB")
print(f"\nTag distribution:")
tag_counts = {}
for chunk in tagged_chunks:
    for tag in chunk['tags']:
        tag_counts[tag] = tag_counts.get(tag, 0) + 1
for tag, count in sorted(tag_counts.items(), key=lambda x: x[1], reverse=True):
    print(f"  {tag}: {count}")

In [ ]:
# Step 4: Implement hybrid retrieval (Semantic + BM25)
from rank_bm25 import BM25Okapi
import numpy as np

tokenized_docs = [doc.lower().split() for doc in documents]
bm25 = BM25Okapi(tokenized_docs)

def hybrid_search(query: str, tag_filter: str = None, top_k: int = 5, alpha: float = 0.5):
    """Hybrid search: semantic (alpha) + BM25 (1-alpha)"""
    
    # Semantic search
    where_filter = {"tags": {"$contains": tag_filter}} if tag_filter else None
    semantic_results = collection.query(
        query_texts=[query],
        n_results=top_k * 2,
        where=where_filter
    )
    
    # BM25 keyword search
    tokenized_query = query.lower().split()
    bm25_scores = bm25.get_scores(tokenized_query)
    
    # Normalize and combine scores
    semantic_ids = semantic_results['ids'][0]
    semantic_distances = semantic_results['distances'][0]
    
    max_dist = max(semantic_distances) if semantic_distances and max(semantic_distances) > 0 else 1
    semantic_scores_norm = {
        semantic_ids[i]: 1 - (semantic_distances[i] / max_dist)
        for i in range(len(semantic_ids))
    }
    
    max_bm25 = max(bm25_scores) if max(bm25_scores) > 0 else 1
    bm25_scores_norm = bm25_scores / max_bm25
    
    combined_scores = {}
    for i, chunk_id in enumerate(ids):
        sem_score = semantic_scores_norm.get(chunk_id, 0)
        bm25_score = bm25_scores_norm[i]
        combined_scores[chunk_id] = alpha * sem_score + (1 - alpha) * bm25_score
    
    top_chunks = sorted(combined_scores.items(), key=lambda x: x[1], reverse=True)[:top_k]
    
    results = []
    for chunk_id, score in top_chunks:
        chunk_idx = int(chunk_id.split('_')[1])
        chunk = tagged_chunks[chunk_idx]
        results.append({
            "chunk_id": chunk_id,
            "score": score,
            "header": chunk['header'],
            "content": chunk['content'],
            "tags": chunk['tags']
        })
    
    return results

print("Hybrid search system ready!")
print("\nTest query: 'research methods'")
test_results = hybrid_search("research methods", tag_filter="<research_methods>", top_k=2)
for i, r in enumerate(test_results, 1):
    print(f"{i}. {r['header'][:60]} (score: {r['score']:.3f})")

In [ ]:
# Step 5: Test Corrective RAG with Document Extraction Query

# Test the corrective RAG system
test_query = "What are the research methods used in this study?"

result = corrective_rag_retrieval(test_query, max_iterations=3, top_k=5)

print(f"\n{'='*80}")
print("CORRECTIVE RAG RESULTS")
print(f"{'='*80}")
print(f"Original Query: {result['query']}")
print(f"Final Query: {result['final_query']}")
print(f"Iterations: {result['iterations']}")
print(f"Relevant Chunks Found: {result['total_found']}")
print(f"\n{'='*80}")

# Now use the relevant chunks to generate the answer
if result['relevant_chunks']:
    context = "\n\n---\n\n".join([chunk['content'][:2000] for chunk in result['relevant_chunks'][:3]])
    
    answer_prompt = f"""Based on the following relevant document sections, answer the user's question comprehensively.

Question: {result['query']}

Relevant Document Sections:
{context}

Provide a detailed answer based only on the information in the document sections above."""
    
    print("\n🤖 Generating answer using LLM...\n")
    
    response = groq_client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[{"role": "user", "content": answer_prompt}],
        temperature=0.2,
        max_completion_tokens=1000,
    )
    
    print(f"{'='*80}")
    print("FINAL ANSWER")
    print(f"{'='*80}")
    print(response.choices[0].message.content)
    print(f"{'='*80}")
else:
    print("\n❌ No relevant chunks found to generate an answer.")

In [ ]:
# Step 6: Complete Document Extraction using Corrective RAG

def extract_document_info_with_corrective_rag():
    """
    Extract all document information using corrective RAG for each section.
    """
    print("\n" + "="*80)
    print("COMPLETE DOCUMENT EXTRACTION WITH CORRECTIVE RAG")
    print("="*80 + "\n")
    
    # Define extraction queries for each section
    queries = {
        'summary': "What is the abstract, main purpose, and overview of this research paper?",
        'methods': "What research methods, study design, data collection, and analytical approaches were used?",
        'findings': "What are the key findings, results, and conclusions of this study?"
    }
    
    extraction_results = {}
    
    # Retrieve relevant chunks for each section using corrective RAG
    for section, query in queries.items():
        print(f"\n{'─'*80}")
        print(f"Extracting: {section.upper()}")
        print(f"{'─'*80}")
        
        result = corrective_rag_retrieval(query, max_iterations=2, top_k=5)
        extraction_results[section] = result
    
    # Compile contexts
    summary_context = "\n\n---\n\n".join([
        chunk['content'][:2500] 
        for chunk in extraction_results['summary']['relevant_chunks'][:3]
    ]) if extraction_results['summary']['relevant_chunks'] else "No relevant information found."
    
    methods_context = "\n\n---\n\n".join([
        chunk['content'][:2500] 
        for chunk in extraction_results['methods']['relevant_chunks'][:3]
    ]) if extraction_results['methods']['relevant_chunks'] else "No relevant information found."
    
    findings_context = "\n\n---\n\n".join([
        chunk['content'][:2500] 
        for chunk in extraction_results['findings']['relevant_chunks'][:3]
    ]) if extraction_results['findings']['relevant_chunks'] else "No relevant information found."
    
    # Generate final extraction
    final_prompt = f"""Based on the retrieved document sections, extract comprehensive information:

## Document Information
**Author(s):** {document_metadata['authors'] or 'Not found'}
**Publication Date:** {document_metadata['date'] or 'Not found'}

## Document Summary
[Provide a 2-3 sentence summary of the paper's main purpose and scope]

## Research Methods
[Summarize:
- Study design and type
- Data sources and databases used
- Sample size and population
- Analytical methods and statistical approaches]

## Key Findings and Conclusions
[Summarize:
- Primary outcomes and results
- Statistical significance of findings
- Main conclusions
- Implications and recommendations]

---

SUMMARY CONTEXT:
{summary_context[:3000]}

---

METHODS CONTEXT:
{methods_context[:3000]}

---

FINDINGS CONTEXT:
{findings_context[:3000]}
"""
    
    print(f"\n{'='*80}")
    print("GENERATING FINAL EXTRACTION...")
    print(f"{'='*80}\n")
    
    response = groq_client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[{"role": "user", "content": final_prompt}],
        temperature=0.2,
        max_completion_tokens=1500,
    )
    
    final_extraction = response.choices[0].message.content
    
    print("="*80)
    print("FINAL DOCUMENT EXTRACTION (CORRECTIVE RAG)")
    print("="*80)
    print(final_extraction)
    print("="*80)
    
    # Print statistics
    print(f"\nExtraction Statistics:")
    print(f"  Summary chunks: {len(extraction_results['summary']['relevant_chunks'])}")
    print(f"  Methods chunks: {len(extraction_results['methods']['relevant_chunks'])}")
    print(f"  Findings chunks: {len(extraction_results['findings']['relevant_chunks'])}")
    print(f"  Total iterations: {sum(r['iterations'] for r in extraction_results.values())}")
    
    return final_extraction

# Run the complete extraction
final_result = extract_document_info_with_corrective_rag()

In [ ]:
# Step 6: Agentic Corrective RAG System
print("=" * 80)
print("AGENTIC CORRECTIVE RAG SYSTEM")
print("=" * 80)

def grade_chunk_relevance(chunk_content: str, query: str) -> dict:
    """
    Grade if a chunk is relevant to the query.
    Returns: {'relevant': bool, 'score': str, 'reason': str}
    """
    grading_prompt = f"""You are a grading expert. Evaluate if the following document chunk is relevant to answer the user's query.

Query: {query}

Document Chunk:
{chunk_content[:1000]}

Respond in this exact format:
RELEVANT: yes or no
SCORE: high, medium, or low
REASON: brief explanation (1 sentence)
"""
    
    try:
        response = groq_client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=[{"role": "user", "content": grading_prompt}],
            temperature=0.1,
            max_completion_tokens=150,
        )
        
        result = response.choices[0].message.content
        
        relevant = False
        score = "low"
        reason = "Unknown"
        
        for line in result.split('\n'):
            if line.startswith('RELEVANT:'):
                relevant = 'yes' in line.lower()
            elif line.startswith('SCORE:'):
                score = line.replace('SCORE:', '').strip().lower()
            elif line.startswith('REASON:'):
                reason = line.replace('REASON:', '').strip()
        
        return {
            'relevant': relevant,
            'score': score,
            'reason': reason
        }
    except Exception as e:
        print(f"Error grading chunk: {e}")
        return {'relevant': True, 'score': 'medium', 'reason': 'Error during grading'}

def rewrite_query(original_query: str, feedback: str = None) -> str:
    """
    Rewrite query to improve retrieval based on feedback.
    """
    if feedback:
        rewrite_prompt = f"""The original query did not retrieve relevant results.

Original Query: {original_query}
Feedback: {feedback}

Rewrite the query to be more specific and improve retrieval. Return only the rewritten query, nothing else."""
    else:
        rewrite_prompt = f"""Improve the following query for better document retrieval by making it more specific and adding relevant keywords.

Original Query: {original_query}

Return only the improved query, nothing else."""
    
    try:
        response = groq_client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=[{"role": "user", "content": rewrite_prompt}],
            temperature=0.3,
            max_completion_tokens=100,
        )
        
        rewritten = response.choices[0].message.content.strip()
        return rewritten
    except Exception as e:
        print(f"Error rewriting query: {e}")
        return original_query

def corrective_rag_retrieval(query: str, max_iterations: int = 3, top_k: int = 5):
    """
    Agentic Corrective RAG system:
    1. Retrieve chunks using hybrid search
    2. Grade relevance of each chunk
    3. If not enough relevant chunks, rewrite query and retry
    4. Iterate until sufficient relevant chunks found or max iterations reached
    """
    print(f"\n{'='*60}")
    print(f"Query: {query}")
    print(f"{'='*60}\n")
    
    current_query = query
    iteration = 0
    all_relevant_chunks = []
    
    while iteration < max_iterations:
        iteration += 1
        print(f"🔄 Iteration {iteration}: Searching with query: '{current_query}'")
        
        # Retrieve chunks using hybrid search
        retrieved_chunks = hybrid_search(current_query, tag_filter=None, top_k=top_k, alpha=0.5)
        
        if not retrieved_chunks:
            print("  ⚠️  No chunks retrieved. Rewriting query...")
            current_query = rewrite_query(current_query, "No results found")
            continue
        
        # Grade each chunk for relevance
        print(f"  📊 Grading {len(retrieved_chunks)} chunks...")
        relevant_count = 0
        
        for chunk in retrieved_chunks:
            grade = grade_chunk_relevance(chunk['content'], query)
            chunk['relevance_grade'] = grade
            
            if grade['relevant'] and grade['score'] in ['high', 'medium']:
                # Avoid duplicates
                if not any(c['chunk_id'] == chunk['chunk_id'] for c in all_relevant_chunks):
                    all_relevant_chunks.append(chunk)
                    relevant_count += 1
                    print(f"    ✅ {chunk['header'][:50]}... - {grade['score'].upper()} ({grade['reason']})")
            else:
                print(f"    ❌ {chunk['header'][:50]}... - Rejected ({grade['reason']})")
        
        print(f"  📈 Found {relevant_count} relevant chunks (Total: {len(all_relevant_chunks)})")
        
        # Check if we have enough relevant chunks
        if len(all_relevant_chunks) >= 3:
            print(f"\n✅ SUCCESS: Found {len(all_relevant_chunks)} relevant chunks!")
            break
        
        # If not enough relevant chunks and we haven't hit max iterations, rewrite query
        if iteration < max_iterations:
            feedback = f"Only found {len(all_relevant_chunks)} relevant chunks. Need more specific information."
            print(f"  🔄 Insufficient relevant chunks. Rewriting query...")
            current_query = rewrite_query(query, feedback)
    
    if not all_relevant_chunks:
        print("\n⚠️  WARNING: No relevant chunks found after all iterations.")
        print("  Falling back to top hybrid search results...")
        all_relevant_chunks = hybrid_search(query, top_k=top_k, alpha=0.5)
    
    return {
        'query': query,
        'final_query': current_query,
        'iterations': iteration,
        'relevant_chunks': all_relevant_chunks,
        'total_found': len(all_relevant_chunks)
    }

print("\nCorrective RAG system ready!\n")